In [1]:
import pandas as pd

In [12]:
# df = pd.read_excel("transfer_out.xlsx")

# df = pd.read_excel("transfer_out_total.xlsx")
# df = pd.read_excel("transfer_out_chengdu.xlsx")

path ="data/origin_data_for_sankey/transfer_out_hangzhou.xlsx"
path ="data/origin_data_for_sankey/transfer_out_nanjing.xlsx"
path ="data/origin_data_for_sankey/transfer_out_hefei.xlsx"
path ="data/origin_data_for_sankey/transfer_out_wuhan.xlsx"
path ="data/origin_data_for_sankey/transfer_out_quanzhou.xlsx"
path ="data/origin_data_for_sankey/transfer_out_xiamen.xlsx"
path ="data/origin_data_for_sankey/transfer_out_guangzhou.xlsx"
df = pd.read_excel(path)

In [13]:
df

,cate_1,cate_2,amt,ratio
0,其他,其他,-,0.00756
1,取款,取款,-,0.04436
2,消费,京东,-,0.00502
3,消费,其他消费,-,0.04747
4,消费,微信,-,0.03424
5,消费,支付宝,-,0.05262
6,消费,美团,-,0.00037
7,理财,他行理财,-,0.01618
8,理财,其他理财,-,0.03834
9,理财,天天基金理财,-,0.00343


In [14]:
df_1 = df.groupby("cate_1", as_index=False)["ratio"].sum().rename(columns={"cate_1":"target", "ratio":"value"})
df_1["source"] = "总资金"

df_2 = df.rename(columns={"cate_1":"source", "cate_2":"target","ratio":"value"})[["source", "target","value"]]

In [15]:
df_1["level"] = 1
df_2["level"] = 2

df_link = pd.concat([df_1, df_2], axis=0)

In [16]:
#df_link.to_csv("transout_sankey.csv", index=False)

In [17]:
colors = { '支付宝理财': "#0066CC",
          '微信': "#00FF00",
          '其他消费': "#66FF00",
          '支付宝': "#3366FF",
          '跨行异名': "#6699CC",
          '微信理财': "#33FF33",
          '美团': "#FFA500",
          '同行同名': "#6B8E23",
          '总资金':"#0099CC",
          '他行理财': "#00CCCC",
          '取款': "#6699FF",
          '第三方转账': "#99CC66",
          '其他理财': "#9999CC",
          '跨行同名': "#99CC00",
          '其他': "#CC9966",
          '京东': "#99CC66",
          '天天基金理财': "#FF9999",
          '理财': "#669999",
          '同行异名': "#6699CC",
          '转账': "#FF9900",
          '行内理财': "#99CC99",
          '购房': "#CCCCFF",
          '还款': "#99CC66",
          '消费': "#33CC66",
         }

In [18]:
## 各个节点的值
pct_1 = df_1.groupby("source")["value"].sum().to_dict()
pct_2 = df_2.groupby("source")["value"].sum().to_dict()
pct_3 = df_2.query("source!=target").groupby("target")["value"].sum().to_dict()

pct = dict()
pct.update(pct_1)
pct.update(pct_2)
pct.update(pct_3)

total = sum(pct_2.values())
pct_new = {n:"{:.2f}%".format(100*(v/total)) if n != "总资金" else "100%" for n, v in pct.items()}


#  构造echart 桑吉图的节点
nodes_ = set(df_1[["source","target"]].values.reshape(-1).tolist()) | set(df_2[["source","target"]].values.reshape(-1).tolist())

nodes = [{"name":n,"itemStyle":{"color":colors[n]}, "value": pct_new[n]} for n in nodes_]

In [19]:
links = df_1.to_dict(orient="records")

links.extend(df_2.query("source!=target").to_dict(orient="records"))

In [20]:
data = {"links": links, "nodes":nodes}

In [21]:
data

{'links': [{'target': '其他', 'value': 0.00756, 'source': '总资金', 'level': 1},
  {'target': '取款', 'value': 0.04436, 'source': '总资金', 'level': 1},
  {'target': '消费', 'value': 0.13972, 'source': '总资金', 'level': 1},
  {'target': '理财', 'value': 0.12009, 'source': '总资金', 'level': 1},
  {'target': '购房', 'value': 0.01215, 'source': '总资金', 'level': 1},
  {'target': '转账', 'value': 0.5830500000000001, 'source': '总资金', 'level': 1},
  {'target': '还款', 'value': 0.09308, 'source': '总资金', 'level': 1},
  {'source': '消费', 'target': '京东', 'value': 0.00502, 'level': 2},
  {'source': '消费', 'target': '其他消费', 'value': 0.04747, 'level': 2},
  {'source': '消费', 'target': '微信', 'value': 0.03424, 'level': 2},
  {'source': '消费', 'target': '支付宝', 'value': 0.05262, 'level': 2},
  {'source': '消费', 'target': '美团', 'value': 0.00037, 'level': 2},
  {'source': '理财', 'target': '他行理财', 'value': 0.01618, 'level': 2},
  {'source': '理财', 'target': '其他理财', 'value': 0.03834, 'level': 2},
  {'source': '理财', 'target': '天天基金理财', 'va

In [14]:
"""
 myChart.setOption(option = {
        tooltip: {
            trigger: 'item',
            triggerOn: 'mousemove'
        },
        series: [
            {
                type: 'sankey',
                emphasis: {
                    focus: 'adjacency'
                },
                nodeAlign: 'left',
                data: data.nodes,
                links: data.links,
                label:{formatter:"{b} {c}"},
                lineStyle: {
                    color: 'source',
                    curveness: 0.5,
                }
            }
        ]
    });
"""

'\n myChart.setOption(option = {\n        tooltip: {\n            trigger: \'item\',\n            triggerOn: \'mousemove\'\n        },\n        series: [\n            {\n                type: \'sankey\',\n                emphasis: {\n                    focus: \'adjacency\'\n                },\n                nodeAlign: \'left\',\n                data: data.nodes,\n                links: data.links,\n                label:{formatter:"{b} {c}"},\n                lineStyle: {\n                    color: \'source\',\n                    curveness: 0.5,\n                }\n            }\n        ]\n    });\n'